In [9]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [19]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_md_testing/tune_logs/MGNN_hyp_small_md_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        params["folder"] = folder
        trials.append((params, eval_res))

In [21]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials: 
    print(trial[0]["folder"])

    print(trial[0])
    print(trial[1]["cummulative_stats"])
    print("---")


Found 17 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_md_testing/tune_logs/MGNN_hyp_small_md_full_mat_loss.py
hyperopt_train_e6b2b_00029_29_batch_size=8,data_aug_factor=3,edge_threshold_val=2.6495,hidden_dim=512,lr=0.0001,message_net_dropou_2025-07-17_01-07-11
{'batch_size': 8, 'data_aug_factor': 3, 'edge_threshold_val': 2.64950666293317, 'grace_epochs': 5, 'hidden_dim': 512, 'loss_on_full_matrix': True, 'lr': 0.00010825336694924005, 'lr_cooldown': 2, 'lr_factor': 0.5, 'lr_min': 1e-06, 'lr_patience': 3, 'lr_threshold': 0.001, 'message_net_dropout': 0.14018093706669849, 'message_net_layers': 3, 'message_passing_steps': 3, 'num_epochs': 30, 'weight_decay': 0.0007128801671222779, 'folder': 'hyperopt_train_e6b2b_00029_29_batch_size=8,data_aug_factor=3,edge_threshold_val=2.6495,hidden_dim=512,lr=0.0001,message_net_dropou_2025-07-17_01-07-11'}
{'energy_abs': {'mean': -1241.855378857725, 'std': 34.536120427237854}, 'iterations': {'mean': 10.9, 'std': 0.4582575694

In [13]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.2f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 10.9(0.5) & -1241.9(34.5) & -0.702(0.009) & 0.210(0.01) & 0.0075(0.0004) \\
1 & 11.0(0.4) & -1238.6(33.8) & -0.700(0.009) & 0.212(0.01) & 0.0076(0.0004) \\
2 & 11.2(0.6) & -1239.9(37.7) & -0.701(0.011) & 0.204(0.01) & 0.0079(0.0005) \\
3 & 11.2(0.4) & -1257.6(38.5) & -0.711(0.010) & 0.222(0.02) & 0.0084(0.0003) \\
4 & 11.4(0.6) & -1282.0(40.4) & -0.725(0.011) & 0.218(0.01) & 0.0081(0.0003) \\
5 & 11.6(1.0) & -1263.7(42.4) & -0.714(0.013) & 0.222(0.02) & 0.0087(0.0005) \\
6 & 11.6(0.9) & -1275.2(38.9) & -0.721(0.012) & 0.223(0.02) & 0.0087(0.0012) \\
7 & 11.8(0.7) & -1279.0(36.9) & -0.723(0.010) & 0.220(0.01) & 0.0083(0.0009) \\
8 & 11.9(0.9) & -1285.8(40.8) & -0.727(0.013) & 0.226(0.02) & 0.0098(0.0012) \\
9 & 12.0(1.1) & -1263.7(41.2) & -0.714(0.013) & 0.224(0.02) & 0.0084(0.0004) \\
\bottomrule
\end{tabular}



In [14]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 512 & 256 & 512 & 128 & 128 & 512 & 512 & 128 & 128 & 512 \\
Batch Size & 8 & 8 & 8 & 16 & 8 & 32 & 32 & 8 & 32 & 32 \\
Data aug. & 3.00 & 4.00 & 2.00 & 4.00 & 3.00 & 3.00 & 3.00 & 1.00 & 4.00 & 3.00 \\
Edge thres. & 2.65 & 3.25 & 3.86 & 3.11 & 2.22 & 3.31 & 3.86 & 3.82 & 3.62 & 2.56 \\
Message p. steps & 3 & 4 & 3 & 3 & 4 & 2 & 2 & 4 & 3 & 3 \\
Message Net Dropout & 0.14 & 0.24 & 0.02 & 0.05 & 0.16 & 0.20 & 0.10 & 0.02 & 0.26 & 0.22 \\
Message Net Layers & 3 & 3 & 4 & 4 & 4 & 2 & 3 & 3 & 3 & 3 \\
Learn Rate & 1.08e-04 & 3.99e-04 & 1.95e-04 & 3.27e-04 & 4.82e-04 & 8.66e-04 & 5.65e-04 & 3.32e-03 & 1.57e-03 & 1.73e-03 \\
Weight Decay & 7.13e-04 & 1.24e-04 & 2.35e-04 & 1.50e-05 & 8.57e-04 & 1.35e-04 & 4.36e-05 & 7.39e-04 & 5.69e-05 & 5.30e-05 \\
\bottomrule
\end{tabular}

